In [ ]:
# uncomment the next line to run it once to install dependencies.
#!pip install dash plotly pandas 'numpy<2.0.0' 'quote_chart>=0.1.2'

In [ ]:
from quote_chart import create_chart_app
import plotly.express as px

def create_figure(x0, x1):
    df = px.data.stocks()
    df.set_index('date', inplace=True)
    return px.line(df['GOOG'])

app = create_chart_app(create_figure)
app.run_server()

In [ ]:
def generate_random_prices():
    # Generate realistic minute data
    np.random.seed(42)

    # Generate datetime index for 30 days of minute data
    dates = pd.date_range('2023-01-01 09:30', periods=30*24*60, freq='min')

    # Initialize data with log returns to simulate realistic price movements
    log_returns = 1 + np.random.normal(0, 0.001, len(dates))
    price = 100 * log_returns.cumprod()  # Starting price of 100

    data = pd.DataFrame(index=dates)
    data['close'] = price
    data['open'] = data['close'].shift(1)

    # Correct data for realism
    data['high'] = data[['open', 'close']].max(axis=1) * (1 + np.abs(np.random.normal(0, 0.001, len(dates))))
    data['low'] = data[['open', 'close']].min(axis=1) * (1 - np.abs(np.random.normal(0, 0.001, len(dates))))
    data['volume'] = np.random.randint(100, 1000, size=len(dates))

    # Round values to two decimal places
    data = data.round(2)
    return data

data = generate_random_prices()

In [ ]:
# resampled_df will keep resampled version of data. by default there is no resampling until a period button is pressed.
resampled_df = data.copy()

# create_figure is called on zoom/pans. it recreates the plot for new x range.
def create_figure(x0, x1):
    if x0 is not None:
        df = resampled_df[x0:x1]
        if len(df) == 0: # case when period is D but current range is less than a day.
            df = resampled_df[-100:]
    else:
        # by default (before zoom/pan) show last 100 candles.
        df = resampled_df[-100:]
    # the rest of the code in this function is regular code for plotly.
    # define multiple panes. The top pane will be for the main price chart with candles. The second pane is for volumes.
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                vertical_spacing=0.01,
                row_heights=[0.8, 0.2],
                specs=[[{"secondary_y": True}], [{"secondary_y": True}]])
    # plot the main chart with price candles.
    fig.add_trace(go.Candlestick(x=df.index, open=df['open'], high=df['high'], low=df['low'], close=df['close'],
                                 name='Prices'), row=1, col=1)
    fig.add_trace(go.Bar(x=df.index, y=df['volume'], name='Volume', marker=dict(color='orange')), row=2, col=1)
    # set the default dragmode to pan, remove the range slider because i use zoom/pan instead of it.
    fig.update_layout(
        dragmode='pan',
        xaxis_rangeslider_visible=False,
        width=1200, # px
        height=600,
        margin=dict(l=3, r=3, t=3, b=3),
        yaxis=dict(side='right'),
        yaxis3=dict(side='right'),
        yaxis5=dict(side='right'),
        legend=dict(y=0.97, x=0.97),
        )
    fig.update_xaxes(
        ticklabelposition="outside right",  # keep labels on the right so that they don't affect margin-left.
    )
    return fig

# called when a period button is pressed under the plot.
def on_period_change(button_id):
    global resampled_df, selected_period
    if button_id == '':
        return
    selected_period = button_id
    resampled_df = data.resample(selected_period).agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'volume': 'sum'
    })
    resampled_df = resampled_df.dropna()
    
# create dash app and run it.
app = create_chart_app(create_figure, on_period_change)
app.run_server(debug=True)

print('Use Shift+scroll to scroll horizontally and Ctrl+scroll to zoom at cursor position.')
